# awk and sed practice

## 1. Intro

Text files preprocessing is quite important for NLP or Text Mining. 

These work includes replacing some particular texts; filtering target records; inserting some fields into texts.

Of course we can use python to do these jobs. Python scripts can be neat and powerful. The best advantage we can get by using python is that the scripts can be run in different platforms. 

However, if we take linux as our production environment, linux's text processing tools such as **awk** and **sed** can be more powerful and productive. We can regard these tools as a text processing architecture. By only adding a few lines, we can get the result we want.

I will introduce the basic idea and usage of **awk** and **sed** by some simple but frequently used practice.

## 2.awk

### 2.1 basic grammer

``` bash
awk [options] 'script' var=value file(s) 
awk [options] -f scriptfile var=value file(s)
```
* -F fs, delimiter
* -f scripfile, we can read awk code from the scriptfile 
* -v var=value, send outter var to awk

**basic strcture**

``` bash
awk 'BEGIN{ print "start" } pattern{ commands } END{ print "end" }' file 
```

First, run BEGIN part once; for every line in the file, do pattern{ commands }; in the end, run END part once.


In [3]:
echo -e "A line 1nA line 2" | awk 'BEGIN{ print "Start" } { print } END{ print "End" }'

Start
A line 1nA line 2
End


### 2.2 Arithmetic operator
+ - 
* / &	
+ - !	
^ ***	
++ --
